In [1]:
import os
from time import time
import yaml

from argparse import ArgumentParser, Namespace

from s3ts.api.nets.wrapper import WrapperModel
from s3ts.helper_functions import load_dm, str_time, get_parser

from pytorch_lightning import Trainer
from pytorch_lightning.utilities.model_summary import summarize

from s3ts.helper_functions import get_model

import torch
import numpy as np

import matplotlib.pyplot as plt

In [2]:
p = get_parser()
args = p.parse_args("--mode img --dataset UCI-HAR --lr 0.001 --subjects_for_test 21 --window_size 32 --window_stride 2 --batch_size 128 --encoder_architecture cnn_gap --encoder_features 20 --decoder_architecture mlp --decoder_features 32 --decoder_layers 1 --pattern_size 32 --rho 0.1 --compute_n 300 --pattern_type fftcoef --num_workers 8 --max_epochs 3 --normalize --reduce_imbalance --training_dir training_cv2 --n_val_subjects 4 --weight_decayL1 0 --weight_decayL2 0".split())

In [3]:
dm = load_dm(args)

# class_changes = [0] + list(np.nonzero(np.diff(dm.stsds.SCS))[0])
# print(len(class_changes), "number of class changes")

Loaded dataset UCI-HAR with a total of 815614 observations for window size 32
Using fft coefficients for the pattern...
Computing dissimilarity frames...
hash of computed patterns: UCI-HAR_1a78a5eb1918c9325882e66582ddd7cb91d05690
Sampling 59084 (balanced) observations per epoch.
Using 708997 observations for training, 106617 for validation and 25512 observations for test


In [46]:
a = np.random.randn(6, 100)

In [48]:
np.mean(a, axis=-1, keepdims=True)

array([[ 0.03215157],
       [-0.04996987],
       [-0.11088493],
       [-0.07243088],
       [-0.01792369],
       [ 0.02863692]])

In [9]:
res = process_fft_frequencies(dm.stsds.STS.numpy(), dm.stsds.SCS.numpy(), np.fft.fftfreq(24)[:24//2])

In [75]:
np.fft.fftfreq(25, d=1)

array([ 0.  ,  0.04,  0.08,  0.12,  0.16,  0.2 ,  0.24,  0.28,  0.32,
        0.36,  0.4 ,  0.44,  0.48, -0.48, -0.44, -0.4 , -0.36, -0.32,
       -0.28, -0.24, -0.2 , -0.16, -0.12, -0.08, -0.04])

In [96]:
res[0][1]

array([-2.85808594e-05-5.41998642e-17j,  4.58712553e-01+5.89340599e-01j,
       -6.77149307e-01-3.58365335e+00j, -7.07050642e-02+4.30176867e+00j,
        7.04246860e-01+1.48860647e-01j,  1.22052063e+00-1.54406083e+00j,
        3.32903567e-01+4.48695709e-01j, -7.90843307e-02+1.94112239e-01j,
       -3.97718251e-01+3.71582323e-01j, -4.07156932e-03+9.15315725e-02j,
        1.51628765e-01-2.23280650e-01j,  6.19594030e-01+3.83596034e-01j])

In [ ]:
ps = 24
l = 5
c = 4
x = np.zeros(ps)
freq = np.fft.fftfreq(ps)[:ps//2]
print(freq)
for i, m in enumerate(res[l][c].real):
    x += m * np.sin(2*np.pi*freq[i]*np.arange(ps))
for i, m in enumerate(res[l][c].imag):
    x += m * np.cos(2*np.pi*freq[i]*np.arange(ps))
    
plt.plot(x)